<a href="https://colab.research.google.com/github/amaljoseph/30SecondVocabulary/blob/master/30SecondVocabulary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting GDrive#

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
root_path =  'gdrive/My Drive/30SecondVocabulary'

In [ ]:
cd My Drive/30SecondVocabulary

[Errno 2] No such file or directory: 'My Drive/30SecondVocabulary'
/content


In [ ]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Random Folder Picker #

In [ ]:
def random_folder_picker():
  random_folder_id = str(np.random.randint(1, 4))
  return random_folder_id

# Image function #

In [ ]:
import  urllib
from google.colab.patches import cv2_imshow

In [ ]:
def get_image_id():
  return str(np.random.randint(250))


def get_blurred_image():  
  id = get_image_id()
  square_url = 'https://picsum.photos/id/'+id+'/500/500/?blur=6.5'
  rectangle_url = 'https://picsum.photos/id/'+id+'/1280/720/?blur=6.5'


  # Square Image
  response = urllib.request.urlopen(square_url)
  image = np.asarray(bytearray(response.read()), dtype="uint8")
  square_img = cv.imdecode(image, cv.IMREAD_COLOR)

  # Rectangle Image
  response = urllib.request.urlopen(rectangle_url)
  image = np.asarray(bytearray(response.read()), dtype="uint8")
  rectangle_img = cv.imdecode(image, cv.IMREAD_COLOR)

  return response.read(), square_img, rectangle_img

# Reading Data From Google Sheet #

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
def get_word_from_gsheet():
  wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1EnzZnT2vMLpludfgpupUS72U5Jh2__FzJvQecWtA-wk/edit#gid=0')
  sheet = wb.get_worksheet(0)
  row = 10
  val=sheet.cell(row,1).value
  return val

get_word_from_gsheet()

'infantile'

# Scraping text from dictionary #

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def make_soup():
  word = get_word_from_gsheet()
  dictionary_url = 'https://dictionary.cambridge.org/dictionary/english/'+word

  page = requests.get(dictionary_url)
  soup = BeautifulSoup(page.content, 'html.parser')

  return soup

In [ ]:
def get_text(soup):
  
  pos = soup.find_all('span', class_='pos dpos')[0].text
  meaning = soup.find_all('div', class_='def ddef_d db')[0].text
  example = soup.find_all('span', class_='eg deg')[0].text
  synonym = soup.find_all('span', class_='x-h dx-h')[0].text

  uk_pron = soup.find(class_='uk dpron-i ').find(class_='pron dpron').find(class_='ipa dipa lpr-2 lpl-1').text
  us_pron = soup.find(class_='us dpron-i ').find(class_='pron dpron').find(class_='ipa dipa lpr-2 lpl-1').text

  return pos, meaning, example, synonym, uk_pron, us_pron

# Audio Extraction #

In [ ]:
def get_audio_url(soup):
  uk_audio = soup.find(class_='uk dpron-i ').find(class_='daud').find(id='ampaudio1').find('source')['src']
  us_audio = soup.find(class_='us dpron-i ').find(class_='daud').find(id='ampaudio2').find('source')['src']

  uk_url = 'https://dictionary.cambridge.org/'+uk_audio
  us_url = 'https://dictionary.cambridge.org/'+us_audio

  return uk_url, us_url

# Image + Audio #

In [ ]:
!pip3 install ffmpeg-python
import ffmpeg

In [ ]:
soup = make_soup()

img=get_blurred_image()[0]
aud=get_audio_url(soup)[0]
input_still = ffmpeg.input(img)
input_audio = ffmpeg.input(aud)

(
    ffmpeg
    .concat(input_still, input_audio, v=1, a=1)
    .output("output.mp4")
    .run()

)

Error: ignored